# 01 — EDA & Data Prep for RUL (Line 10 + Line 20)

This notebook:
- Loads raw Line 10 & 20 datasets
- Normalizes key columns
- Builds a Remaining Useful Life (RUL) label
- Saves a combined `rul_labeled` dataset to `data/processed/`.

In [1]:
from pathlib import Path
import sys, json, os
import pandas as pd
import numpy as np
import yaml
from datetime import timedelta
NB_PATH = Path.cwd()

def find_project_root(start: Path) -> Path:
    cur = start
    for _ in range(6):
        if (cur / "requirements.txt").exists() or (cur / "configs").exists() or (cur / "data").exists():
            return cur
        cur = cur.parent
    try:
        i = [p.name.lower() for p in start.parents].index("notebooks")
        return start.parents[i+1]
    except ValueError:
        return start

ROOT = find_project_root(NB_PATH)
DATA_DIR = ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
for p in [DATA_DIR, RAW_DIR, PROCESSED_DIR]:
    p.mkdir(parents=True, exist_ok=True)
CONFIGS_DIR = ROOT / 'configs'

print("ROOT:", ROOT)
print("DATA_DIR:", DATA_DIR)

ROOT: d:\Richard Files\WORK\pdm-project
DATA_DIR: d:\Richard Files\WORK\pdm-project\data


In [2]:
# --- Load configs
def load_yaml(p):
    import yaml, io
    with open(p, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

base_cfg   = load_yaml(CONFIGS_DIR / 'base.yaml')
task_cfg   = load_yaml(CONFIGS_DIR / 'task_cof.yaml')
line10_cfg = load_yaml(CONFIGS_DIR / 'line10.yaml') if (CONFIGS_DIR / 'line10.yaml').exists() else None
line20_cfg = load_yaml(CONFIGS_DIR / 'line20.yaml') if (CONFIGS_DIR / 'line20.yaml').exists() else None

base_cfg, task_cfg

({'random_seed': 42,
  'time_index': 'timestamp',
  'id_col': 'machine_id',
  'target_rul': 'RUL',
  'target_cof': 'CoF',
  'freq': '1min'},
 {'task': 'CoF',
  'metrics': ['f1', 'recall', 'roc_auc'],
  'train_val_test_split': {'method': 'time_series_split', 'n_splits': 3}})

**IMPORT DATASET FROM SQL**

In [3]:
from dotenv import load_dotenv
import os
import pyodbc

# Load .env
load_dotenv()

# Ambil variabel dari .env
server     = os.getenv("MSSQL_SERVER")
port       = os.getenv("MSSQL_PORT")
database   = os.getenv("MSSQL_DATABASE")
username   = os.getenv("MSSQL_USERNAME")
password   = os.getenv("MSSQL_PASSWORD")
driver     = os.getenv("MSSQL_DRIVER")
encrypt  = os.getenv("MSSQL_ENCRYPT")
trust    = os.getenv("MSSQL_TRUST_SERVER_CERTIFICATE")

# Bangun connection string
conn_str = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Encrypt={encrypt};"
    f"TrustServerCertificate={trust};"
)

# Connect
conn = pyodbc.connect(conn_str, timeout=500)
print("Connected!")


Connected!


In [3]:
line10_path = pd.read_sql("SELECT * FROM dbo.DM_Machine_Learning_Line_10", conn)

C:\Users\albert.dammara\AppData\Local\Temp\ipykernel_18668\1596707094.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  line10_path = pd.read_sql("SELECT * FROM dbo.DM_Machine_Learning_Line_10", conn)


In [4]:
line10_path.head()

,Timestamp,Mesin,Energy_Sectors.Extruder_10_Target,EXT_10.Extruder_Load,EXT_10.Machine_Hour,EXT_10.Machine_Run,EXT_10.Melt_Press,EXT_10.Melt_Temp_1,EXT_10.Motor_Extruder_Run,EXT_10.Panel_Humidity,...,PM_Extruder_10.V_avg,PM_Extruder_10.W_tot,Start_Time,End_Time,Level_1,Level_2,Level_3,Detail,rn,Breakdown
0,2025-01-01 00:02:00,10,None,None,None,None,None,None,None,None,...,None,None,NaT,NaT,None,None,None,None,1,0
1,2025-01-01 00:06:00,10,None,None,None,None,None,None,None,None,...,None,None,NaT,NaT,None,None,None,None,1,0
2,2025-01-01 00:25:00,10,None,None,None,None,None,None,None,None,...,None,None,NaT,NaT,None,None,None,None,1,0
3,2025-01-01 02:08:00,10,None,None,None,None,None,None,None,None,...,None,None,NaT,NaT,None,None,None,None,1,0
4,2025-01-01 02:31:00,10,None,None,None,None,None,None,None,None,...,None,None,NaT,NaT,None,None,None,None,1,0


In [4]:
df10 = pd.read_sql("SELECT * FROM dbo.DM_Machine_Learning_Line_10", conn)
df20 = pd.read_sql("SELECT * FROM dbo.DM_Machine_Learning_Line_20", conn)

dfs=[df10, df20]
df_raw = pd.concat(dfs, ignore_index=True)
print("Combined raw:", df_raw.shape)
df_raw.head()

C:\Users\albert.dammara\AppData\Local\Temp\ipykernel_23500\2330132286.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df10 = pd.read_sql("SELECT * FROM dbo.DM_Machine_Learning_Line_10", conn)
C:\Users\albert.dammara\AppData\Local\Temp\ipykernel_23500\2330132286.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df20 = pd.read_sql("SELECT * FROM dbo.DM_Machine_Learning_Line_20", conn)


Combined raw: (699840, 220)


,Timestamp,Mesin,Energy_Sectors.Extruder_10_Target,EXT_10.Extruder_Load,EXT_10.Machine_Hour,EXT_10.Machine_Run,EXT_10.Melt_Press,EXT_10.Melt_Temp_1,EXT_10.Motor_Extruder_Run,EXT_10.Panel_Humidity,...,EXT_20_Cincinnati.Temp_Setpoint_Dies_16,EXT_20_Cincinnati.Vacuum_Press_Actual,EXT_20_Cincinnati.Vacuum_Press_Setpoint,EXT_20_Cincinnati.Vacuum_Pump_Run,PM_Extruder_20.A_avg,PM_Extruder_20.Frequency,PM_Extruder_20.P,PM_Extruder_20.Power_Factor,PM_Extruder_20.V_avg,PM_Extruder_20.W_tot
0,2025-01-01 00:02:00,10,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-01 00:06:00,10,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-01 00:25:00,10,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-01 02:08:00,10,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-01 02:31:00,10,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**USE BELOW IF PULLING DATA DIRECTLY FROM LOCAL MACHINE (DATA/RAW)**

In [4]:
# Adjust these if your filenames/locations are different
line10_path = pd.read_sql("SELECT * FROM dbo.DM_Machine_Learning_Line_10", conn)
##line20_path = RAW_DIR / "Line20" / "DM_Machine_Learning_Line_20.csv"

dfs = []

if line10_path.exists():
    df10 = pd.read_csv(line10_path)
    df10["__line"] = 10
    dfs.append(df10)
    print("Loaded Line 10:", df10.shape, "from", line10_path)
else:
    print("Line 10 file not found:", line10_path)

if line20_path.exists():
    df20 = pd.read_csv(line20_path)
    df20["__line"] = 20
    dfs.append(df20)
    print("Loaded Line 20:", df20.shape, "from", line20_path)
else:
    print("Line 20 file not found:", line20_path)

if not dfs:
    raise FileNotFoundError("No raw CSVs found for Line 10 or Line 20.")

df_raw = pd.concat(dfs, ignore_index=True)
print("Combined raw:", df_raw.shape)
df_raw.head()

C:\Users\albert.dammara\AppData\Local\Temp\ipykernel_11004\3195966576.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  line10_path = pd.read_sql("SELECT * FROM dbo.DM_Machine_Learning_Line_10", conn)


AttributeError: 'DataFrame' object has no attribute 'exists'

**CONTINUE HERE AFTER CHOOSING TO EITHER PULL FROM SERVER / PULL FROM LOCAL MACHINE**

In [6]:
# Normalize key columns
df = df_raw.copy()

# Detect possible columns
machine_candidates = [c for c in df.columns if "mesin" in c.lower() or "machine" in c.lower()]
time_candidates     = [c for c in df.columns if "timestamp" in c.lower() or "waktu" in c.lower() or c.lower()=="time"]
breakdown_candidates = [c for c in df.columns if "breakdown" in c.lower() or "failure" in c.lower()]

print("Machine candidates:", machine_candidates)
print("Time candidates:", time_candidates)
print("Breakdown candidates:", breakdown_candidates)

# --- Machine ID: choose the FIRST occurrence (or refine later)
if len(machine_candidates) == 0:
    raise RuntimeError("No machine_id-like column found!")
elif len(machine_candidates) > 1:
    print("⚠️ Multiple machine-like columns found. Using:", machine_candidates[0])
df = df.rename(columns={machine_candidates[0]: "machine_id"})

# Drop all other machine-like columns if they exist
for c in machine_candidates[1:]:
    df = df.drop(columns=[c])

# --- Timestamp
if len(time_candidates) == 0:
    raise RuntimeError("No timestamp-like column found!")
df = df.rename(columns={time_candidates[0]: "timestamp"})
for c in time_candidates[1:]:
    df = df.drop(columns=[c])

# --- Breakdown
if len(breakdown_candidates) > 0:
    df = df.rename(columns={breakdown_candidates[0]: "Breakdown"})
    for c in breakdown_candidates[1:]:
        df = df.drop(columns=[c])
    df["Breakdown"] = pd.to_numeric(df["Breakdown"], errors="coerce").fillna(0).astype(int)
else:
    print("⚠️ No breakdown-like column found")

df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.sort_values(["machine_id", "timestamp"]).reset_index(drop=True)

df[["machine_id","timestamp"] + (["Breakdown"] if "Breakdown" in df.columns else [])].head()


Machine candidates: ['Mesin', 'EXT_10.Machine_Hour', 'EXT_10.Machine_Run', 'EXT_20.Machine_Hour', 'EXT_20.Machine_Run', 'EXT_20_Cincinnati.Machine_Hour', 'EXT_20_Cincinnati.Machine_Run']
Time candidates: ['Timestamp']
Breakdown candidates: ['Breakdown']
⚠️ Multiple machine-like columns found. Using: Mesin


,machine_id,timestamp,Breakdown
0,10,2025-01-01 00:00:00,0
1,10,2025-01-01 00:01:00,0
2,10,2025-01-01 00:02:00,0
3,10,2025-01-01 00:03:00,0
4,10,2025-01-01 00:04:00,0


In [8]:
# Compute RUL in minutes until next Breakdown==1 per machine
if "Breakdown" not in df.columns:
    raise RuntimeError("Need 'Breakdown' to compute RUL. Adapt this cell if you use another label source.")

ruls = []

for mid, g in df.groupby("machine_id", sort=True):
    g = g.sort_values("timestamp").reset_index(drop=True)
    ts = g["timestamp"].values
    br = g["Breakdown"].values

    idx_fail = np.where(br == 1)[0]
    if len(idx_fail) == 0:
        rul = np.full(len(g), np.nan, dtype="float32")
    else:
        rul = np.full(len(g), np.nan, dtype="float32")
        for fi in idx_fail:
            t_fail = ts[fi]
            dt = (t_fail - ts[:fi+1]) / np.timedelta64(1, "m")
            dt = np.maximum(dt, 0)
            mask = np.isnan(rul[:fi+1])
            rul[:fi+1][mask] = dt[mask]

    g2 = g.copy()
    g2["RUL"] = rul
    ruls.append(g2)

df_rul = pd.concat(ruls, ignore_index=True)
df_rul = df_rul.dropna(subset=["RUL"]).reset_index(drop=True)

print("RUL-labeled shape:", df_rul.shape)
df_rul[["machine_id","timestamp","Breakdown","RUL"]].head(20)

RUL-labeled shape: (656762, 215)


,machine_id,timestamp,Breakdown,RUL
0,10,2025-01-01 00:00:00,0,17520.0
1,10,2025-01-01 00:01:00,0,17519.0
2,10,2025-01-01 00:02:00,0,17518.0
3,10,2025-01-01 00:03:00,0,17517.0
4,10,2025-01-01 00:04:00,0,17516.0
5,10,2025-01-01 00:05:00,0,17515.0
6,10,2025-01-01 00:06:00,0,17514.0
7,10,2025-01-01 00:07:00,0,17513.0
8,10,2025-01-01 00:08:00,0,17512.0
9,10,2025-01-01 00:09:00,0,17511.0


In [9]:
LABELLED_RUL_PARQ = PROCESSED_DIR / "rul_labeled.parquet"
##LABELLED_RUL_CSV  = PROCESSED_DIR / "rul_labeled.csv"

df_rul.to_parquet(LABELLED_RUL_PARQ, index=False)
##df_rul.to_csv(LABELLED_RUL_CSV, index=False)

print("Saved:", LABELLED_RUL_PARQ)
##print("Saved:", LABELLED_RUL_CSV)

df_rul["RUL"].describe()

Saved: d:\Richard Files\WORK\pdm-project\data\processed\rul_labeled.parquet


count    656762.000000
mean      98113.843750
std       80791.867188
min           0.000000
25%       31409.250000
50%       78749.500000
75%      147599.000000
max      297119.000000
Name: RUL, dtype: float64